In [5]:
import pandas as pd
import numpy as np
import pyodbc
import datetime
import os
import re

In [6]:
#get today's date
today=datetime.datetime.now()
#create folder paramaters
folder="C:\\Users\\bedlamini\\Documents\\VAT Schedules\\Format\\"

extracted="C:\\Users\\bedlamini\\Documents\\VAT Schedules\\Extracted\\"

error="C:\\Users\\bedlamini\\Documents\\VAT Schedules\\Error\\"

In [8]:
files=[]
count=0
for file in os.listdir(folder):
    file_path= os.path.join(folder, file)
    if os.path.isfile(file_path):
        
        full_name = os.path.basename(file_path)
        file_name = os.path.splitext(full_name)
        files.append(file_path)
        print(file_name)
        try:
            df=pd.read_excel(file_path,sheet_name=["VAT schedule form--input tab","VAT schedule form--output tab"],header=2)
            input_df=df.get("VAT schedule form--input tab")
            output_df=df.get("VAT schedule form--output tab")
            PTIN = re.search(r"\d{9}", file).group()
            FILESPLIT=re.split(r"\s",file,3)
            YEAR=FILESPLIT[1]
            PERIOD=FILESPLIT[2]
            #PTIN=pd.read_excel(file_path,sheet_name="VAT schedule form--input tab",usecols="B",nrows=1,header=None,names=["Value"]).iloc[0]["Value"]
            #Period=pd.read_excel(file_path,sheet_name="VAT schedule form--input tab",usecols="J",nrows=1,header=None,names=["Value"]).iloc[0]["Value"]
            
        except:
            print("ERROR!!!!!!!!!!!")
            print("Moving File to error folder......")
            os.rename(file_path, error + os.path.basename(file_path))
            print("Moved")
            print("NEXT.......")
            print("")
            count = count + 1
            continue
        try:
                      
            input_df.columns=["Date","TIN","Supplier","invoice","Description","Exempt","Zero","Unregistered","Standard_domestic","Standard_imports","Adjustment","Adjustment_other","VAT","Total"]
            
            output_df.columns=["Date","TIN","Customer","invoice","Description","Exempt","Zero_exports","Zero_supplies","Standard_rated","Adjustment","Adjustment_other","VAT","Total"]
            #input_df["Date"]=pd.to_datetime(input_df["Date"],format='%Y%m%d')
            #output_df["Date"]=pd.to_datetime(output_df["Date"],format='%Y%m%d')
            input_df["Date"]=pd.to_datetime(input_df["Date"],format='%Y%m%d',errors='coerce')
            output_df["Date"]=pd.to_datetime(output_df["Date"],format='%Y%m%d',errors='coerce')
            
            
                      
            
            input_df["invoice"]=input_df["invoice"].astype(str)
            output_df["invoice"]=output_df["invoice"].astype(str)
            
            input_df["Exempt"]=input_df["Exempt"].astype(float)
            output_df["Exempt"]=output_df["Exempt"].astype(float)
            
            input_df["Zero"]=input_df["Zero"].astype(float)
            output_df["Zero_exports"]=output_df["Zero_exports"].astype(float)
            
            input_df["Unregistered"]=input_df["Unregistered"].astype(float)
            output_df["Zero_supplies"]=output_df["Zero_supplies"].astype(float)
            
            input_df["Standard_domestic"]=input_df["Standard_domestic"].astype(float)
            output_df["Standard_rated"]=output_df["Standard_rated"].astype(float)
            
            input_df["Standard_imports"]=input_df["Standard_imports"].astype(float)
            output_df["Adjustment"]=output_df["Adjustment"].astype(float)
            
            input_df["Adjustment"]=input_df["Adjustment"].astype(float)
            output_df["Adjustment_other"]=output_df["Adjustment_other"].astype(float)
            
            input_df["VAT"]=input_df["VAT"].astype(float)
            output_df["VAT"]=output_df["VAT"].astype(float)
            
            input_df["Total"]=input_df["Total"].astype(float)
            output_df["Total"]=output_df["Total"].astype(float)
            
            input_df["Adjustment"]=input_df["Adjustment"].astype(float)
            
            
            
            #input_df=input_df[input_df["Date"].notnull() & input_df["Supplier"].notnull() & input_df["invoice"].notnull()]  
            #output_df= output_df[output_df["Customer"].notnull()& output_df["Date"].notnull()
                          #& output_df["invoice"].notnull()]
            input_df=input_df[input_df["Date"].notnull() & input_df["Supplier"].notnull()]  
            output_df= output_df[output_df["Customer"].notnull()& output_df["Date"].notnull()]
            
                       
            input_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)
            output_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)
            
            input_df = input_df.fillna(value=0)
            output_df = output_df.fillna(value=0)
        except:
            print(str(e))
            print("ERROR CLEANING DATAFRAME")
            print("Moving File to error folder......")
            os.rename(file_path, error + os.path.basename(file_path))
            print("Moved")                   
            print("NEXT.......")
            print("")
            count = count + 1
            continue
        
        else:
            conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=srv-bi-dev;'
                      'Database=ERS_data_raw;'
                      'Trusted_Connection=yes;')

            cursor = conn.cursor()

            for index, row in input_df.iterrows():
                #print(row.Exempt,row.Zero,row.VAT,row.Total,row.Unregistered,row.Standard_domestic,row.Standard_imports,row.Adjustment,row.Adjustment_other)
                
                cursor.execute("INSERT INTO dbo.Refund_VAT_InputFormat3 (date,TIN,supplier_name,invoice,particulars,exempt,zero_rated,vat,total,ptin,created_date,unregistered,standard_rated,standard_rated_imports,Period,Adjustment,Adjustment_other,Year) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", 
                row.Date, str(row.TIN), str(row.Supplier),row.invoice,row.Description,row.Exempt,row.Zero,row.VAT,row.Total,str(PTIN),today,row.Unregistered,row.Standard_domestic,row.Standard_imports,PERIOD,row.Adjustment,row.Adjustment_other,str(YEAR))
            #print("")
            for index, row in output_df.iterrows():
                #print(row.Exempt,row.Zero_exports,row.VAT,row.Total,row.Zero_supplies,row.Standard_rated,row.Adjustment,row.Adjustment_other)
                cursor.execute("INSERT INTO dbo.Refund_VATOutputFormat3 (date,TIN,customer_name,invoice,particulars,exempt,zero_rated,vat,total,ptin,CreateDate,zero_rated_supplies,standard_rated,adjustments,adjustments_other,Period,Year) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
                row.Date, str(row.TIN), str(row.Customer),str(row.invoice),row.Description,row.Exempt,row.Zero_exports,row.VAT,row.Total,str(PTIN),today,row.Zero_supplies,row.Standard_rated,row.Adjustment,row.Adjustment_other,PERIOD,str(YEAR))
            conn.commit()
            os.rename(file_path, extracted + os.path.basename(file_path))
            cursor.close()
            print("DONE")
            print("NEXT")
       
    print("ALL FILES PROCESSED")  
        
    
                               

('100111042 2023 September sep 23 vat', '.xlsx')


C:\Users\bedlamini\AppData\Local\Temp\ipykernel_22692\419194485.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)
C:\Users\bedlamini\AppData\Local\Temp\ipykernel_22692\419194485.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output_df["TIN"]= input_df["TIN"].astype(str).str.rstrip('0').str.replace('.','',regex=True)


DONE
NEXT
ALL FILES PROCESSED
